In [1]:
import numpy as np
import tensorflow as tf
import os

In [2]:
LOGDIR = "unit_logs/"

In [3]:
log_writer = tf.summary.FileWriter(LOGDIR)

In [4]:
class DNC:
    def __init__(self,input_size,output_size,seq_len,num_words,word_size,read_heads,device = '/device:GPU:1'):
        self.output_size = output_size
        self.controller_output_size = output_size
        self.read_heads = read_heads
        self.word_size = word_size
        self.num_words = num_words
        
        with tf.device(device):
            #controller
            
            self.read_weightings = tf.Variable( tf.fill([num_words,read_heads],1e-8,name = "read_weights_wr_init"),trainable=False,name="read_weights_wr")
            self.read_vectors = tf.Variable(tf.fill([read_heads,word_size],1e-8,name="init_read_vectors"),trainable=False,name="read_vectors_r")
            
            self.write_weightings = tf.Variable( tf.fill([num_words,1],1e-8,name = "write_weights_wr_init"),trainable=False,name="write_weights_wr")
            self.write_vector = tf.Variable(tf.fill([1,word_size],1e-8,name="init_write_vectors"),trainable=False,name="write_vector_v")
            
            self.memory_matrix = tf.Variable( tf.truncated_normal([num_words,word_size]),trainable=False,name = "memory_matrix_M")
            
            with tf.name_scope("calc_output_vector_y/"):
                self.read_vectors_weights = tf.get_variable("Wr",shape=[self.read_heads*self.word_size,self.output_size],initializer=tf.contrib.layers.xavier_initializer())
            
            with tf.name_scope("train_data/inputs_x"):        
                
                
                self.input_x = tf.placeholder(tf.float32,shape=[1,input_size],name = "input_x")
                
            with tf.name_scope("train_data/outputs_y"):    
                self.output_y = tf.placeholder(tf.float32,shape=[1,output_size],name="output_y")
            
            

            with tf.variable_scope("feed_forward/layer1"):
                self.weights1 = tf.get_variable("weights1",shape=[input_size,32],initializer=tf.contrib.layers.xavier_initializer())
                self.biases1 = tf.get_variable("biases1",shape=[32],initializer=tf.zeros_initializer())
                layer1_activation = tf.nn.relu(tf.matmul(self.input_x,self.weights1)+self.biases1)
            with tf.variable_scope("feed_forward/layer2"):
                self.weights2 = tf.get_variable("weights2",shape=[32,output_size],initializer=tf.contrib.layers.xavier_initializer())
                self.biases2 = tf.get_variable("biases2",shape=[output_size])
                layer2_activation = tf.nn.relu(tf.matmul(layer1_activation,self.weights2)+self.biases2)
                self.output_logits = layer2_activation
                
                
                
            
            with tf.name_scope("calc_output_vector_y/controller_output_vt"): #will need to be changed when final output is coded
                self.output_vector_weights = tf.get_variable("Wy",shape=[self.controller_output_size,self.output_size],initializer=tf.contrib.layers.xavier_initializer())
                self.controller_output_vector = tf.matmul(layer2_activation,self.output_vector_weights,name="controller_output_vector_vt")
                
                
                
            with tf.name_scope("update_write_weightings/"):
                self.write_keys = tf.truncated_normal([1,word_size],name = "temp_random_write_key")
                self.write_strength = self.one_plus(tf.truncated_normal([1,1],name = "temp_random_write_strength"),name="write_strengths_one_plus")
                self.erase_vector = tf.nn.sigmoid(tf.truncated_normal([1,word_size],name="temp_random_erase_vector"),name="erase_vector_sigmoid")
                
                self.allocation_gate = tf.nn.sigmoid(tf.truncated_normal([1,1],name="temp_random_allocation_gate"),name="allocation_gate_sigmoid")
                self.write_gate = tf.nn.sigmoid(tf.truncated_normal([1,1],name="temp_random_write_gate"),name="write_gate_sigmoid")
                
                self.write_content_weighting = self.content_lookup(self.write_keys,self.write_strength,"write") 
                
                
                new_write_weightings = self.write_gate*((1-self.allocation_gate)*self.write_content_weighting)
                self.write_weightings = tf.assign(self.write_weightings,new_write_weightings,name ="update_write_weightings")  
                
            with tf.name_scope("calc_write_vector/"):
                self.write_vector = tf.assign(self.write_vector,tf.truncated_normal([1,word_size],name="temp_write_vector"))
                
                
            with tf.name_scope("update_memory_matrix"):
                new_memory_matrix = self.memory_matrix*(1-tf.matmul(self.write_weightings,self.erase_vector)) + tf.matmul( self.write_weightings,self.write_vector)
                self.memory_matrix = tf.assign(self.memory_matrix,new_memory_matrix,name="update_memory_matrix")
            
            with tf.name_scope("update_read_weightingss/"):
                self.read_keys = tf.truncated_normal([read_heads,word_size],name="temp_random_read_keys")
                self.read_strengths = self.one_plus(tf.truncated_normal([read_heads,1],name = "temp_random_read_strengths"),name="read_strengths_one_plus")
                
            
            #TODO: multiply by read_mode
                self.content_weigthing = self.content_lookup(self.read_keys,self.read_strengths,"read") #N*1
                self.read_weightings = tf.assign(self.read_weightings, self.content_weigthing,name ="update_read_weightings")
            
            with tf.name_scope("calc_read_vectors/"):
                self.read_vectors = tf.assign(self.read_vectors, tf.transpose(tf.matmul(self.memory_matrix,self.read_weightings,transpose_a=True)),name  ="update_read_vectors")
            
            with tf.name_scope("calc_output_vector_y/weighted_read_vectors"):
                weighted_read_vectors = tf.matmul(tf.reshape(self.read_vectors,[1,self.read_heads*self.word_size]),self.read_vectors_weights,name = "weighted_read_vectors")
                
            
            with tf.name_scope("calc_output_vector_y/"):
                self.output_vector = tf.add( self.controller_output_vector  , weighted_read_vectors , name="output_vector_y")
            
            
            with tf.name_scope("loss"):    
                self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.output_vector, labels=self.output_y))
        
            with tf.device('/device:CPU:0'):
                tf.summary.scalar("cross_entropy_loss",self.loss)
                
            with tf.name_scope("optimizer"):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(self.loss)
 
    def one_plus(self,x,name=""):
        return  tf.add( 1.0,tf.log(tf.expand_dims(tf.add(1.0,tf.exp(x)),0)),name=name)
    
    def content_lookup(self,key,key_strength,mode="write"):
        with tf.name_scope(mode+"cosine_content_lookup"):
            normalized_memory = tf.nn.l2_normalize(self.memory_matrix,1) #N*W
            normalized_key = tf.nn.l2_normalize(key,0) #1*W
        
        
            z = tf.matmul(normalized_memory,normalized_key,transpose_a=False,transpose_b=True)
            content_lookup_result = tf.reshape(tf.nn.softmax(z*tf.squeeze(key_strength),0),[self.num_words,self.read_heads])
        
        # N*1
        return  content_lookup_result

In [5]:
def train():
    #generate the input output sequences, randomly intialized
    tf.reset_default_graph()
    num_seq = 10
    seq_len = 6
    seq_width = 4
    iterations = 600
    con = np.random.randint(0, seq_width,size=seq_len)
    seq = np.zeros((seq_len, seq_width))
    seq[np.arange(seq_len), con] = 1
    end = np.asarray([[-1]*seq_width])
    zer = np.zeros((seq_len, seq_width))
    
    j = 0
    
    debug_every = 1
    final_i_data = np.concatenate((seq, zer), axis=0)
    final_o_data = np.concatenate((zer, seq), axis=0)
    with tf.Session() as session:
        
        dnc = DNC(input_size=seq_width, output_size=seq_width, seq_len=seq_len, num_words=20, word_size=5, read_heads=1)
        log_writer.add_graph(session.graph)
        
        session.run(tf.global_variables_initializer())
        feed_dict = {dnc.input_x : np.reshape(final_i_data[j],[1,seq_width]), dnc.output_y: np.reshape(final_o_data[j],[1,seq_width])}
        
        merged_summary = tf.summary.merge_all()
        rk,mm,cw = session.run([dnc.read_keys,dnc.memory_matrix,dnc.content_weigthing],feed_dict=feed_dict)
        print(rk,mm,cw)
        #print(start_mem)
        ol,l,op,summ,mm = session.run([dnc.output_vector,dnc.loss,dnc.optimizer,merged_summary,dnc.read_keys], feed_dict=feed_dict)
        
        log_writer.add_summary(summ)
        #print(l)
        #print(mm)

In [6]:
train()

[[ 0.39497289 -0.82659078  1.77593958 -1.23148978  0.59110743]] [[-1.48690271 -0.24559009  1.32002068 -0.61622483  0.19444641]
 [-0.19670375 -0.96104103 -0.07071274 -0.87835163 -1.9464916 ]
 [-0.33569089  0.05019461 -0.60443139  1.05275834  0.22924083]
 [ 1.43154204  1.70968616  0.74253392 -0.53904504  0.09785389]
 [-0.03216974  0.018766    1.48649657 -1.15948033 -0.89035749]
 [-0.63967073 -1.61195278 -0.427618    0.59085381 -0.18877554]
 [ 0.9726395   1.60470283  0.86295801  0.42248476  1.20804763]
 [ 0.20940953 -0.24888763 -0.1277148  -0.50363731 -1.34205472]
 [-0.08086148 -0.42332864  0.46347427  1.59289575 -0.83522516]
 [ 0.05782478 -0.15293057 -1.05900228 -0.85203379  1.88221967]
 [ 0.67070061 -0.86690456 -1.2280581  -0.10816561  1.04694688]
 [-1.36626101 -0.53223974  0.1692007   0.21239825 -1.06225777]
 [-1.03696978  0.24237126 -1.21351171 -0.58721548  0.3362208 ]
 [-0.9008314   1.20902419 -0.53596032 -0.79741406  0.82041115]
 [-0.43407959  0.65325016  1.04456627 -0.04720911 -1.1

In [7]:
def create_graph():
    with tf.device('/device:GPU:1'):
        var = tf.Variable(tf.zeros([2,1]),trainable = False)
    
        var2 = tf.Variable(tf.ones([2,1]),trainable=False)
        
        var = tf.assign(var,tf.add(var,var2))
        
        return var,var2
    

In [8]:
def train():    
    tf.reset_default_graph()
    with tf.Session() as session:
        var,var2 = create_graph()
        session.run(tf.global_variables_initializer())
        
        for i in range(5):
            result = session.run(var)
            print(result)

In [9]:
train()

[[ 1.]
 [ 1.]]
[[ 2.]
 [ 2.]]
[[ 3.]
 [ 3.]]
[[ 4.]
 [ 4.]]
[[ 5.]
 [ 5.]]


In [10]:
def content_lookup(memory_matrix,key,key_strength):
        with tf.name_scope("cosine_content_lookup"):
            normalized_memory = tf.nn.l2_normalize(memory_matrix,1) #N*W
            normalized_key = tf.nn.l2_normalize(key,0) #1*W
        
        
            z = tf.matmul(normalized_memory,normalized_key,transpose_a=False,transpose_b=True)
            content_lookup_result = tf.reshape(tf.nn.softmax(z*tf.squeeze(key_strength),0),[20,1])
            #content_lookup_result = tf.nn.softmax(tf.losses.cosine_distance(normalized_memory, tf.nn.l2_normalize(key, 1), dim=0))
            #content_lookup_result = normalized_memory
        # N*1
        return  content_lookup_result

def one_plus(self,x,name=""):
        return  tf.add( 1.0,tf.log(tf.expand_dims(tf.add(1.0,tf.exp(x)),0)),name=name)

def create_graph():
    memory_matrix = tf.truncated_normal([20,5])
    key  = tf.truncated_normal([1,5],name="temp_random_read_keys")
    #key_strength = one_plus( x=tf.truncated_normal([1,1],name = "temp_random_read_strengths"),name="op")
    key_strength = tf.truncated_normal([1,1],name = "temp_random_read_strengths")
    key_strength_op = one_plus( key_strength,name = "temp_random_read_strengths")
        
        
    result = content_lookup(memory_matrix,key,key_strength_op)
    
    return memory_matrix,key,key_strength,key_strength_op,result

def train():    
    tf.reset_default_graph()
    with tf.Session() as session:
        memory_matrix,key,key_strength,key_stren_op,result = create_graph()
        session.run(tf.global_variables_initializer())
        
        
        mm,k,ks,ks_op,r = session.run([memory_matrix,key,key_strength,key_stren_op,result])
        print("mem",mm)
        print("key",k)
        print("key stren",ks)
        print("key stren op",ks_op)
        print("resul",np.sum(r))
        print("----")
        print(1 + np.log(1+np.exp(ks)))
train()

TypeError: one_plus() missing 1 required positional argument: 'x'

In [ ]:
1 + np.log(1+np.exp(-1.06015229))